In [97]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [168]:
df = pd.read_csv('ESS8_data.csv')

In [169]:
X = df[['SD1', 'PO1', 'UN1', 'AC1', 'SC1',
       'ST1', 'CO1', 'TR1', 'HD1', 'AC2', 'SC2', 'ST2',
       'CO2', 'PO2', 'BE2', 'TR2', 'HD2']].iloc[range(3),:]
X = X.to_numpy().T
N, M = X.T.shape
K = 3

In [170]:
np.random.seed(42)

p = np.random.permutation(range(N))[:K]
Z = X[:,p]
# Z = np.random.randint(1, 6, (M,K)) 

#Q = Z.T @ Z
#Qt = torch.tensor(Q,requires_grad=False).float()

R = X.T @ X
Rt = torch.tensor(R,requires_grad=False).float()

In [171]:
# applies both for eq. 7 and 8
def error(a_i,Qt,qt):
    return (0.5 * a_i.T @ Qt @ a_i) - (qt.T @ a_i)

In [191]:
torch.manual_seed(42)

A = np.zeros((K,N))#.tolist()
B = np.zeros((N,K))#.tolist()

# LOOP UNTIL RSS IS LOW
for n in range(10): #N
    Q = Z.T @ Z
    Qt = torch.tensor(Q,requires_grad=False).float()
    
    # LOOP THROUGH ENTIRE A
    for i in range(N):
        q = Z.T @ X[:,i]
        qt = torch.tensor(q,requires_grad=False).float()

        a_i = torch.autograd.Variable(torch.rand(K, 1), requires_grad=True) # eller er det Kx1 ?

        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(a_i,Qt,qt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(10000): # 100000
            Delta = error(a_i,Qt,qt) 
            L = torch.norm(Delta, p=2)
            L.backward()
            a_i.data -= step_size * a_i.grad.data # step
            a_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break

        A[:,i] = np.array(a_i.tolist()).flatten() 
        # print('Loss after: %s' % (torch.norm( error(a_i,Qt,qt) )))

    ### apply softmax here? ###
    
    Z = X @ A.T @ np.linalg.inv(A@A.T)
    
    # LOOP THROUGH ENTIRE B
    for i in range(K): #K
        r = X.T @ Z[:,i]
        rt = torch.tensor(r,requires_grad=False).float()

        b_i = torch.autograd.Variable(torch.rand(N, 1), requires_grad=True) 

        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(b_i,Rt,rt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(10000): # 100000
            Delta = error(b_i,Rt,rt)
            L = torch.norm(Delta, p=2)
            L.backward()
            b_i.data -= step_size * b_i.grad.data # step
            b_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break

        B[:,i] = np.array(b_i.tolist()).flatten() 
        #print('Loss after: %s' % (torch.norm( error(b_i,Rt,rt) )))    
    
    Z = X @ B
    
    Zt = torch.tensor(Z, requires_grad=False).float()
    At = torch.tensor(A, requires_grad=False).float()
    Xt = torch.tensor(X,requires_grad=False).float()
    print("RSS at n=%s" % n, torch.norm(Xt-Zt@At,p='fro'))

Loss before: tensor(4.1257, grad_fn=<NormBackward0>)
Loss is tensor(4.1257, grad_fn=<NormBackward0>) at iteration 0
Loss before: tensor(47.6795, grad_fn=<NormBackward0>)
Loss is tensor(47.6795, grad_fn=<NormBackward0>) at iteration 0
Loss before: tensor(109.4622, grad_fn=<NormBackward0>)
Loss is tensor(109.4622, grad_fn=<NormBackward0>) at iteration 0
Loss before: tensor(174.1082, grad_fn=<NormBackward0>)
Loss is tensor(174.1082, grad_fn=<NormBackward0>) at iteration 0
Loss before: tensor(369.6698, grad_fn=<NormBackward0>)
Loss is tensor(369.6698, grad_fn=<NormBackward0>) at iteration 0
Loss before: tensor(47.2404, grad_fn=<NormBackward0>)
Loss is tensor(47.2404, grad_fn=<NormBackward0>) at iteration 0
RSS at n=0 tensor(29.1799)
Loss before: tensor(103.3252, grad_fn=<NormBackward0>)
Loss is tensor(103.3252, grad_fn=<NormBackward0>) at iteration 0
Loss before: tensor(43.9097, grad_fn=<NormBackward0>)
Loss is tensor(43.9097, grad_fn=<NormBackward0>) at iteration 0
Loss before: tensor(100

In [178]:
A[:,i]

TypeError: list indices must be integers or slices, not tuple

In [180]:
Y = np.zeros((2,3))

In [182]:
Y[:,1] = np.array([1,1])

In [183]:
Y

array([[0., 1., 0.],
       [0., 1., 0.]])

In [41]:
A = torch.rand(2, 2, requires_grad=False)
b = torch.rand(2, 1,  requires_grad=False)

In [87]:
torch.inverse(A)

tensor([[ 2.1062, -1.4374],
        [-2.7958,  3.7027]])